In [1]:
using Gurobi, JuMP   #JuMP是一个优化库(可以在里面插入优化的库，比如Gurobi) Import everything 
using DataFrames, GLM
using Plots
using Distributions
using Random
using LinearAlgebra
using DelimitedFiles

In [9]:
# Multivariable Newsvendor Problem
function GenData(n,m,a,a2,b,b2,c,c2,r,r2,s,form="newsvendor",seed=1)
	Random.seed!(seed)   #Random: Support for generating random numbers.   Random.rand!-Function是的，(seed)可以将随机数固定
	#Drawing random decision data
	x = rand(n)  #x是[0,1]之间的随机数，所以我们产生的data points是随机的
    z = rand(n)
	#Drawing random outcome data
	#y,y_min,y_max = Outcome(x,a,b,c,r,s,form )
    y,y_min,y_max = Outcome(x,z,a,a2,b,b2,c,c2,r,r2,s,form)     
	#Setting all possible values for decisions
    d_1 = collect(0.1:1/m:0.9)     ### How to decide the step size here ????  Make each interval have data points and avoid boundary issues
    d_2 = collect(0.1:1/m:0.9)
    d = []
    d2 = []
    for i = 1:length(d_1)
        num = d_1[i]   # The number we want to push in this iteration
        for j = 1:length(d_1)
            push!(d,num)   # We need to push this number for length(d) times in this iteration
        end
    end
            
    for j = 1:length(d_1)
        append!(d2,d_2)   # We use append() to add a list at the end of the former list
    end
            
        
	return x,z,y,y_min,y_max,d,d2
end

#Function for simulating outcome. 拟合各种函数结果
function Outcome(x,z,a,a2,b,b2,c,c2,r,r2,s,form="newsvendor")
	n = length(x)
    y = zeros(n)
    #Simulating the random outcome for each random decision
    for i = 1:n
        if form == "flat"
            y[i] = a + s*randn()[1]
        elseif form == "flat-double"
            y[i] = a + s*randn()[1]
            if 0.5 <= x[i] <= 0.75
                y[i] = r*a + s*randn()[1]   #这里有随机项，所以对于相同的x值，y值也有很小的差别
            end
        elseif form == "linear"
            y[i] = a + b*x[i] + s*randn()[1]
        elseif form == "linear-double"
            y[i] = a + b*x[i] + s*randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + a*(1-r)*x[i] + s*randn()[1]
            end
        elseif form == "quadratic"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]
        elseif form == "quadratic-double"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]   #此处需要数字而不是数组，故需要将randn() 转化为randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + (3*a*(1-r)+2*b+c)*x[i] + (-2*a*(1-r)-2*b-c)*x[i]^2 + s*randn()[1]    #r具体是多少？如何使得函数在边界点处相等？可以用公式计算出r的表达式
            end
        elseif form == "newsvendor"
        	demand = a + s*randn()[1]
            demand2 = a2 + s*rand()[1]
            # We have new variables in newsvendor problem
        	y[i] = -(c*x[i] + r*b*max(demand-x[i],0) + b*max(x[i]-demand,0) + c2*z[i] + r2*b2*max(demand2-z[i],0) + b2*max(z[i]-demand2,0))
        elseif form == "supplychain-normal"
        	y[i] = x[i] * (quantile(Normal(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        elseif form == "supplychain-pareto"
        	y[i] = x[i] * (quantile(Pareto(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        end		       
    end
    #Scaling the outcome to unit scale
    y_min = minimum(y)
    y_max = maximum(y)
    y = (y - ones(n)*y_min)./(ones(n)*(y_max-y_min))   #将y归一化
	return y,y_min,y_max
end


Outcome (generic function with 2 methods)

In [10]:
# Multivariable Newsvendor Problem (Function for computing result  )
function Result(y_min,y_max,x,z,a,a2,b,b2,c,c2,r,r2,form="newsvendor")
	#Computing the expected outcome for given decision
    if form == "flat"
        y = a
    elseif form == "flat-double"
        y = a
        if 0.5 <= x <= 0.75
            y = r*a               ### What is the value of r??????
        end
    elseif form == "linear"
        y = a + b*x
    elseif form == "linear-double"
        y = a + b*x
        if 0.5 <= x
            y = r*a + a*(1-r)*x
        end
    elseif form == "quadratic"
        y = a + b*x + c*x^2
    elseif form == "quadratic-double"
        y = a + b*x + c*x^2
        if 0.5 <= x
            y = r*a + (3*a*(1-r)+2*b+c)*x + (-2*a*(1-r)-2*b-c)*x^2
        end
    elseif form == "newsvendor"
    	demand = a
        demand2 = a2
    	y = -(c*x + r*b*max(demand-x,0) + b*max(x-demand,0) + c2*z + r2*b2*max(demand2-z,0) + b2*max(z-demand2))
    elseif form == "supplychain-normal"
        y = x * (quantile(Normal(c,r),(a-x)/(a+b)))
    elseif form == "supplychain-pareto"
        y = x * (quantile(Pareto(c,r),(a-x)/(a+b)))
    end
    #Scaling the outcome to unit scale
    y = (y-y_min)/(y_max-y_min)
	return y
end

Result (generic function with 2 methods)

In [4]:
#Function for jointly estimating and optimizing
function EWO(x,Z_data,y,d,d2,kappa,lambda,M,showoutput=false)   # Where do we define x and d here????
    n = Int(length(x))   # Number of the data points
    #n = length(Z)
    m = Int(sqrt(length(d)))  # One dimension of binary variable
    #Defining optimization model
	model = Model()
	set_optimizer(model, Gurobi.Optimizer)      #调用Gurobi里面的optimizer? Yep
	set_optimizer_attributes(model, "OutputFlag" => 0)   #??????   If the value=0, it will output the model, or it will not output the model
    #println("Its correct 1!")
    @variable(model, z[1:m,1:m], Bin)   
    @variable(model, a)     ### 定义变量语法（general way）
    @variable(model, b)
    #@variable(model, c)
    @variable(model, b2)
    @variable(model, u[1:n], Bin)   #u[1:n]都是binary variable
    @variable(model, v[1:n], Bin)
    @variable(model, w[1:n], Bin)
    @variable(model, f[1:n], Bin)   #f[1:n]都是binary variable
    @variable(model, g[1:n], Bin)
    @variable(model, h[1:n], Bin)
    #@variable(model, z[1:m,1:m], Int)   
    #println("Its correct 2!")
    @variable(model, s[1:m,1:m] >= 0)
    @variable(model, t[1:n] >= 0)
    @variable(model, Z[1:m], Bin)
    @variable(model, Z2[1:m], Bin)
	#Defining objective
    @objective(model, Max, sum(s[i,j] for i=1:m for j=1:m) - lambda * sum(t[i] for i=1:n))   # Updated
    #Defining constraints
    # Task: Redo the constraint section!!!
    @constraints(model, begin
        decision, sum(z[i,j] for i=1:m for j=1:m) == 1   #decision here is the name of the constraint Updated
        revenue_passive[i=1:m,j=1:m], s[i,j] <= a+b*d[i]+b2*d2[j]  #Updated
        revenue_active[i=1:m,j=1:m], s[i,j] <= M*z[i,j]   #Updated
        data[i=1:n], u[i] >= v[i] + w[i] - 1   #Updated
        Z_value[k=1:m], Z[k]==sum(z[k,i] for i=1:m)   #Updated
        Z2_value[j=1:m], Z2[j]==sum(z[d,j] for d=1:m)   #Updated
        data_upper[i=1:n], x[i] - sum(d[k]*Z[k] for k = 1:m ) >= kappa - M*v[i]   ### Updated Meaning of this constraints? It's a linear programming problem, but the equivalents are hard to write
        data_lower[i=1:n], sum(d[k]*Z[k] for k = 1:m ) - x[i] >= kappa - M*w[i]   #Updated
        #Z_value[k=1:m], Z[k]==sum(z[k,i] for i=1:m)   #Updated
        #Z2_value[j=1:m], Z2[j]==sum(z[d,j] for d=1:m)   #Updated
        data2[i=1:n], f[i] >= g[i] + h[i] - 1   #Updated
        data2_upper[i=1:n], Z_data[i] - sum(d2[j]*Z2[j] for j=1:m) >= kappa - M*g[i]   ### Updated Meaning of this constraints? It's a linear programming problem, but the equivalents are hard to write
        data2_lower[i=1:n], sum(d2[j]*Z2[j] for j=1:m) - Z_data[i] >= kappa - M*h[i]   #Updated
        estimation_upper[i=1:n], t[i] >= y[i]-a-b*x[i]-b2*Z_data[i]-M*(1-u[i]+1-f[i])   #Updated
        estimation_lower[i=1:n], t[i] >= -y[i]+a+b*x[i]+b2*Z_data[i]-M*(1-u[i]+1-f[i])   #Updated
    end)

    #Optimizing model
    optimize!(model)
    obj_val = objective_value(model)
    time_val = solve_time(model)
    a_val = value.(a)
    b_val = value.(b)
    #c_val = value.(c)
    b2_val = value(b2)
    z_val = value.(z)
    return obj_val,time_val,a_val,b_val,b2_val,z_val
end

EWO (generic function with 2 methods)

In [11]:
#Selecting setup
#setup = ["flat-double",100,0,0,4,1,0.05,1]
#setup = ["linear-double",100,100,0,0.75,1,0.05,1]
#setup = ["quadratic-double",100,100,-100,0.5,1,0.05,0.1]
num_para = 5   # We have 5 parameter combinations
setup[1] = ["newsvendor",0.3,0.4,1,4,1,1,0.75,0.75,0.1,0.05,0.1]  # rho: the degree of scaling
setup[2] = ["newsvendor",0.3,0.5,1,3,1,1,0.75,0.75,0.1,0.05,0.1]  # rho: the degree of scaling
setup[3] = ["newsvendor",0.3,0.5,1,4,1,2,0.75,0.75,0.1,0.05,0.1]  # rho: the degree of scaling
setup[4] = ["newsvendor",0.3,0.5,1,4,1,1,0.75,0.5,0.1,0.05,0.1]  # rho: the degree of scaling
setup[5] = ["newsvendor",0.3,0.5,1,4,1,1,0.75,0.75,0.1,0.05,0.1]  # rho: the degree of scaling
#setup = ["supplychain-normal",1,1,4,3,0.1,0.05,0.1]
#setup = ["supplychain-pareto",1,1.5,0.25,1,0.1,0.05,0.1]

for i = 1:5 # We have five parameter combinations
#Setting parameters    
    form_data = setup[i][1]
    alpha = setup[i][2]
    alpha2 = setup[i][3]
    beta = setup[i][4]
    beta2 = setup[i][5]
    gamma = setup[i][6]
    gamma2 = setup[i][7]
    rho = setup[i][8]
    rho2 = setup[i][9]
    sigma = setup[i][10]   #相当于前面的s
    kappa = setup[i][11]
    lambda = setup[i][12]
    # The following parameters do not changes in all combinations 
    n = 1000   # We have one thousand data points
    m = 20   # We have m strategies( Change number of strategies from 45 to 20 in order to save time)
    big_M = 1000
    seed_numbers = 100   # Generate 100 datasets
end

In [12]:
#Printing parameter settings
println("Parameter settings:")
println("form_data: ", form_data)
println("alpha: ", alpha)
println("alpha2:", alpha2)
println("beta: ", beta)
println("beta2: ", beta2)
println("gamma: ", gamma)
println("gamma2: ", gamma2)
println("rho: ", rho)
println("rho2: ", rho2)
println("sigma: ", sigma)
println("kappa: ", kappa)
println("lambda: ", lambda)
println("n: ", n)
println("m: ", m)
println("seed_numbers: ", seed_numbers)

Parameter settings:


LoadError: UndefVarError: form_data not defined

In [23]:
    #Initializing results table
    #Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 15+6)    #创建一个空数组。为啥有四种变量？？？？？？
using Printf
setup = []   # Initialize the setup list
push!(setup,["newsvendor",0.3,0.4,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,3,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,2,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.5,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
println(setup)
num_para = 5   # We have five parameter combinations
ave_y = []   # This vector store the average y values for each parameter settings

for k=1:num_para
    # using printf macro with @ sign
    @printf("Now we begin %d th parameter setting", k)
    form_data = setup[k][1]
    alpha = setup[k][2]
    alpha2 = setup[k][3]
    beta = setup[k][4]
    beta2 = setup[k][5]
    gamma = setup[k][6]
    gamma2 = setup[k][7]
    rho = setup[k][8]
    rho2 = setup[k][9]
    sigma = setup[k][10]   #相当于前面的s
    kappa = setup[k][11]
    lambda = setup[k][12]
    # The following parameters do not changes in all combinations 
    n = 1000   # We have one thousand data points
    m = 20   # We have m strategies( Change number of strategies from 45 to 20 in order to save time)
    big_M = 1000
    seed_numbers = 50   # Generate 100 datasets
    
    seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
    #Running models for specified number of seeds to generate results
    Sum = 0
    for seed_number = 1:seed_numbers
        println()   #空行，显得更美观
        println("seed_number: ", seed_number)

        #Initializing parameter settings in results table
        #Results[3*(seed_number-1)+1,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]   
        #Results[3*(seed_number-1)+2,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]
        #Results[3*(seed_number-1)+3,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]

        #Generating data
        x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)

        #Solving Estimate-While-Optimize model
        solution_EWO = EWO(x_data,z_data,y_data,d_data,d2_data,kappa,lambda,big_M)

        #Saving solution and plotting parameters for Estimate-While-Optimize model
        a_EWO = solution_EWO[3]
        b_EWO = solution_EWO[4]
        b2_EWO = solution_EWO[5]
        z_EWO = solution_EWO[6]   # The value of z_EWO, which is a matrix
        x_EWO = 0   # Define x_EWO
        Z_EWO = 0   # Define Z_EWO
        for i=1:length(z_EWO)   # Iterate all items in z_EWO and find the corresponding x_EWO and Z_EWO
            if z_EWO[i] ==1   # Note the index
                if rem(i,9)==0
                    index_1 = 9
                else
                    index_1 = i - fld(i,9)*9   # fld: floor the result
                end
                #index_1 = i - fld(i,9)*9   # fld: floor the result
                index_2 = cld(i,9)   # cld: ceil the result
                x_EWO = x_EWO + d_data[index_2]
                Z_EWO = Z_EWO + d2_data[index_1]   # Change needed!!
        
                break
            end
        end

        y_EWO = Result(y_min_data,y_max_data,x_EWO,Z_EWO,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,form_data)  

        #Writing Estimate-While-Optimize results to results table
        #Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

        #Printing Estimate-While-Optimize results
        println("EWO")
        println("a: ", a_EWO)
        println("b: ", b_EWO)
        #println("c: ", c_EWO)
        println("b2: ",b2_EWO)
        println("x: ", x_EWO)
        println("z: ",Z_EWO)
        println("y: ", y_EWO)
        Sum = Sum + y_EWO

    end
    #Writing results table to text file
    #writedlm("Results2.txt", Results)
    ### If wanna plot, delete for and end, make the seed_number a fixed number

    #Plotting data   Why cannot plot here?Because our seed number is changing at first, so the software does not know how to draw it 
    push!(ave_y,Sum/seed_numbers)
    #ave_y = Sum/1
    println("The average y value is: ",Sum/seed_numbers)
    println()
    println()
    println()
end
    
writedlm("Result.txt",ave_y)
    #=
    gr()
    p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
    p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")      ###Why sometimes plot, sometimes plot!??????
    plot(p)
    =#

Any[Any["newsvendor", 0.3, 0.4, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 3, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 2, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.5, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1]]
Now we begin 1 th parameter setting
seed_number: 1
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0x325c12f6
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 646 rows and 646 columns
Presolve time: 0.98s
Presolved: 7967 rows, 69

seed_number: 4
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0x76561243
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [7e-05, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 2e+03]
Presolve removed 608 rows and 608 columns
Presolve time: 1.00s
Presolved: 8005 rows, 7007 columns, 85903 nonzeros
Variable types: 1292 continuous, 5715 integer (5714 binary)
Found heuristic solution: objective 300.6862365

Root relaxation: objective 1.000000e+03, 3119 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000  

seed_number: 7
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0xfdc7d23c
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 608 rows and 608 columns
Presolve time: 0.92s
Presolved: 8005 rows, 7007 columns, 85771 nonzeros
Variable types: 1292 continuous, 5715 integer (5714 binary)
Found heuristic solution: objective 13.9717287

Root relaxation: objective 1.000000e+03, 2973 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000   

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0xaa71085f
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 610 rows and 610 columns
Presolve time: 1.02s
Presolved: 8003 rows, 7005 columns, 85800 nonzeros
Variable types: 1292 continuous, 5713 integer (5712 binary)
Found heuristic solution: objective 8.0511357
Found heuristic solution: objective 100.3428463

Root relaxation: objective 1.000000e+03, 3020 iterations, 0.18 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node 

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0xb6dcd707
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 2e+03]
Presolve removed 612 rows and 612 columns
Presolve time: 0.91s
Presolved: 8001 rows, 7003 columns, 85962 nonzeros
Variable types: 1292 continuous, 5711 integer (5710 binary)
Found heuristic solution: objective 300.3127488

Root relaxation: objective 1.000000e+03, 3264 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0   80  300.3

...

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0x2fae3202
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 2e+03]
Presolve removed 572 rows and 572 columns
Presolve time: 0.95s
Presolved: 8041 rows, 7043 columns, 85816 nonzeros
Variable types: 1292 continuous, 5751 integer (5750 binary)
Found heuristic solution: objective 10.6045622

Root relaxation: objective 1.000000e+03, 3288 iterations, 0.16 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0  107   10.60

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0xd05cd730
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 558 rows and 558 columns
Presolve time: 0.87s
Presolved: 8055 rows, 7057 columns, 86088 nonzeros
Variable types: 1292 continuous, 5765 integer (5764 binary)
Found heuristic solution: objective 400.2333688
Found heuristic solution: objective 400.6279812

Root relaxation: objective 1.000000e+03, 2992 iterations, 0.14 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

seed_number: 1
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0x867785ce
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 646 rows and 646 columns
Presolve time: 0.95s
Presolved: 7967 rows, 6969 columns, 85258 nonzeros
Variable types: 1292 continuous, 5677 integer (5676 binary)
Found heuristic solution: objective 12.3637820
Found heuristic solution: objective 300.3102583

Root relaxation: objective 1.000000e+03, 3005 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8613 rows, 7615 columns and 92635 nonzeros
Model fingerprint: 0xdb4e0583
Variable types: 1292 continuous, 6323 integer (6323 binary)
Coefficient statistics:
  Matrix range     [7e-05, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 2e+03]
Presolve removed 608 rows and 608 columns
Presolve time: 0.96s
Presolved: 8005 rows, 7007 columns, 85903 nonzeros
Variable types: 1292 continuous, 5715 integer (5714 binary)
Found heuristic solution: objective 300.6796468

Root relaxation: objective 1.000000e+03, 3119 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0   92  300.6

Excessive output truncated after 525414 bytes.

In [25]:
#Initializing results table
#Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 15+6)    #创建一个空数组。为啥有四种变量？？？？？？
using Printf
setup = []   # Initialize the setup list
push!(setup,["newsvendor",0.3,0.4,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,3,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,2,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.5,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
println(setup)
num_para = 5   # We have five parameter combinations
ave_y = []   # This vector store the average y values for each parameter settings

for k=4:num_para
    # using printf macro with @ sign
    @printf("Now we begin %d th parameter setting", k)
    form_data = setup[k][1]
    alpha = setup[k][2]
    alpha2 = setup[k][3]
    beta = setup[k][4]
    beta2 = setup[k][5]
    gamma = setup[k][6]
    gamma2 = setup[k][7]
    rho = setup[k][8]
    rho2 = setup[k][9]
    sigma = setup[k][10]   #相当于前面的s
    kappa = setup[k][11]
    lambda = setup[k][12]
    # The following parameters do not changes in all combinations 
    n = 1000   # We have one thousand data points
    m = 20   # We have m strategies( Change number of strategies from 45 to 20 in order to save time)
    big_M = 1000
    seed_numbers = 50   # Generate 100 datasets
    
    seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
    #Running models for specified number of seeds to generate results
    Sum = 0
    for seed_number = 1:seed_numbers
        println()   #空行，显得更美观
        println("seed_number: ", seed_number)

        #Initializing parameter settings in results table
        #Results[3*(seed_number-1)+1,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]   
        #Results[3*(seed_number-1)+2,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]
        #Results[3*(seed_number-1)+3,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]

        #Generating data
        x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)

        #Solving Estimate-While-Optimize model
        solution_EWO = EWO(x_data,z_data,y_data,d_data,d2_data,kappa,lambda,big_M)

        #Saving solution and plotting parameters for Estimate-While-Optimize model
        a_EWO = solution_EWO[3]
        b_EWO = solution_EWO[4]
        b2_EWO = solution_EWO[5]
        z_EWO = solution_EWO[6]   # The value of z_EWO, which is a matrix
        x_EWO = 0   # Define x_EWO
        Z_EWO = 0   # Define Z_EWO
        for i=1:length(z_EWO)   # Iterate all items in z_EWO and find the corresponding x_EWO and Z_EWO
            if z_EWO[i] ==1   # Note the index
                if rem(i,9)==0
                    index_1 = 9
                else
                    index_1 = i - fld(i,9)*9   # fld: floor the result
                end
                #index_1 = i - fld(i,9)*9   # fld: floor the result
                index_2 = cld(i,9)   # cld: ceil the result
                x_EWO = x_EWO + d_data[index_2]
                Z_EWO = Z_EWO + d2_data[index_1]   # Change needed!!
        
                break
            end
        end

        y_EWO = Result(y_min_data,y_max_data,x_EWO,Z_EWO,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,form_data)  

        #Writing Estimate-While-Optimize results to results table
        #Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

        #Printing Estimate-While-Optimize results
        println("EWO")
        println("a: ", a_EWO)
        println("b: ", b_EWO)
        #println("c: ", c_EWO)
        println("b2: ",b2_EWO)
        println("x: ", x_EWO)
        println("z: ",Z_EWO)
        println("y: ", y_EWO)
        Sum = Sum + y_EWO

    end
    #Writing results table to text file
    #writedlm("Results2.txt", Results)
    ### If wanna plot, delete for and end, make the seed_number a fixed number

    #Plotting data   Why cannot plot here?Because our seed number is changing at first, so the software does not know how to draw it 
    push!(ave_y,Sum/seed_numbers)
    #ave_y = Sum/1
    println("The average y value is: ",Sum/seed_numbers)
    println()
    println()
    println()
end
    
writedlm("Result.txt",ave_y)
    #=
    gr()
    p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
    p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")      ###Why sometimes plot, sometimes plot!??????
    plot(p)
    =#

Any[Any["newsvendor", 0.3, 0.4, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 3, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 2, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.5, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1]]
Now we begin 4 th parameter setting
seed_number: 1
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1001.5236170876391
b: -10.915443770527016
b2: -1.7282908423452048
x: 0.1
z: 0.5
y: 0.6729867086514932

seed_number: 2
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 997.5188240944219
b: -0.8511824439844674
b2: 2.8514379444183775
x: 0.15
z: 0.1
y: 0.9168067457588005

seed_number: 3
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1001.0359518448852
b: -10.149184405248024
b2: -0.21033404360373165
x: 0.1
z: 0.45
y: 0.7392605996149989

seed_number: 4
Academic license - fo

seed_number: 42
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1000.9527146012645
b: -9.337289220159319
b2: -0.18985679248598858
x: 0.1
z: 0.3
y: 0.8000766165186123

seed_number: 43
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1001.2502538124738
b: -4.709082086688157
b2: -0.9168771809471069
x: 0.15
z: 0.5
y: 0.6771669314805447

seed_number: 44
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1000.9388392137232
b: -9.192301452152586
b2: -0.13072712338591067
x: 0.1
z: 0.4
y: 0.7387673508206483

seed_number: 45
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 999.8676839546888
b: 0.648205136992645
b2: 0.33747765805942764
x: 0.1
z: 0.15
y: 0.8813416000867424

seed_number: 46
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 999.823796458323
b: 0.6097029244951112
b2: 0.4609329969098326
x: 0.1
z: 0.15
y: 0.8920321257262521

seed_number: 47
Academic license - 

Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1000.9354417705002
b: 0.3078295754895449
b2: -1.2882996373988742
x: 0.15
z: 0.1
y: 0.9213040752700347

seed_number: 36
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 997.304840728673
b: -0.48032240351654676
b2: 3.047990568531863
x: 0.15
z: 0.1
y: 0.9171026423457476

seed_number: 37
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1001.4908017939528
b: -0.06421164919076432
b2: -14.843806290337147
x: 0.1
z: 0.3
y: 0.8569571075008815

seed_number: 38
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1000.9704544420539
b: 0.1422222325072439
b2: -1.0940851836717376
x: 0.15
z: 0.1
y: 0.9373490181809748

seed_number: 39
Academic license - for non-commercial use only - expires 2023-08-31
EWO
a: 1001.7546156033513
b: -12.414221946636399
b2: -1.1404297970904462
x: 0.1
z: 0.25
y: 0.893574923300708

seed_number: 40
Academic license - for non-comm

The average y value is: 0.870264974005419    [^Parameter setting one]  

The average y value is: 0.876895249953845    [^Parameter setting two] 

The average y value is: 0.8354793435983696    [^Parameter setting three]  

The average y value is: 0.8187510870405957  [^Parameter setting four ] 

The average y value is: 0.8671201496619356   [^Parameter setting five]
 

In [5]:
using DelimitedFiles
number = [1,2,3,4,5]
writedlm("Numbers.txt", number)

In [3]:
a = [1:2  4:5  7:8]
println(length(a))
println(a)

6
[1 4 7; 2 5 8]


# Benchmark Models

# Linear Regression and Quadratic Regression

In [13]:
setup = []   # Initialize the setup list
push!(setup,["newsvendor",0.3,0.4,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,3,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,2,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.5,0.1,0.05,0.1])  # rho: the degree of scaling
push!(setup,["newsvendor",0.3,0.5,1,4,1,1,0.75,0.75,0.1,0.05,0.1])  # rho: the degree of scaling
println(setup)
num_para = 5   # We have five parameter combinations
seed_number = 1 # Initialize the variable seed_number
lin_max = []   # The list stores the maximum value of linear prediction for each dataset
qua_max = []
ave_lin = []   # Store the average maximum value from each parameter setting
ave_qua = []
for k=1:num_para   # Iterate all parameter settings
    # using printf macro with @ sign
    println("Now parameter setting number is: ", k)
    form_data = setup[k][1]
    alpha = setup[k][2]
    alpha2 = setup[k][3]
    beta = setup[k][4]
    beta2 = setup[k][5]
    gamma = setup[k][6]
    gamma2 = setup[k][7]
    rho = setup[k][8]
    rho2 = setup[k][9]
    sigma = setup[k][10]   #相当于前面的s
    kappa = setup[k][11]
    lambda = setup[k][12]
    # The following parameters do not changes in all combinations 
    n = 1000   # We have one thousand data points
    m = 20   # We have m strategies( Change number of strategies from 45 to 20 in order to save time)
    big_M = 1000
    seed_numbers = 50   # Generate 100 datasets
    
    seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
    #Running models for specified number of seeds to generate results
    for seed_number = 1:seed_numbers 
        println()
        println("seed number:", seed_number)
        # Two-Covariate Linear Regression
        x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
        # Combine three columns of data together
        com_data = hcat(x_data,z_data,y_data)
        df = DataFrame(x_data = com_data[:,1],z_data = com_data[:,2],y_data = com_data[:,3])   # convert the data to DataFrame format
        # Train test split
        using Lathe.preprocess: TrainTestSplit
        train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
        #println(test)
        #Train the model
        lin_mol = @formula(y_data ~  0 + x_data + z_data)
        linearRegressor = lm(lin_mol, train)   # Train the LR model from the trained dataset
        # Prediction
        # Delete rows that contain illegal elements first
        tbl = Tables.rowtable(test)
        i = 0
        del_list = []   # Store the rows that should be deleted
        for row in Tables.rows(tbl)
            i = i+1
            if ((row[1]<0.1 || row[1]>0.9) || (row[2]<0.1 || row[2]>0.9))
                push!(del_list,i)
            end
        end
        delete!(test, del_list) #Delete rows that contain illegal elements  
        ypredicted_test = predict(linearRegressor, test)
        lin_val = maximum(ypredicted_test)
        push!(lin_max,lin_val)
    
    
    
        x2 = x_data.^2
        #println(x2)
        z2 = z_data.^2
        xz = x_data.*z_data
        com_data = hcat(x_data,z_data,x2,xz,z2,y_data)
        df2 = DataFrame(x_data = com_data[:,1],z_data = com_data[:,2],x2 = com_data[:,3],xz = com_data[:,4],z2 = com_data[:,5],y_data = com_data[:,6])   # convert the data to DataFrame format
        #print(df2)
        # Train test split
        using Lathe.preprocess: TrainTestSplit
        train2, test2 = TrainTestSplit(df2,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
        #Train the model
        #print(train2)
        lin_mol2 = @formula(y_data ~  0 + x_data + z_data + x2 + xz + z2)
        #println(lin_mol2)
        #println("The model is correct1!")
        linearRegressor2 = lm(lin_mol2, train2)
        #println("The model is correct2!")
        # Delete rows that contain illegal elements first
        tbl2 = Tables.rowtable(test2)
        j = 0
        del_list2 = []   # Store the rows that should be deleted
        for row in Tables.rows(tbl2)
            j = j+1
            if ((row[1]<0.1 || row[1]>0.9) || (row[2]<0.1 || row[2]>0.9))
                push!(del_list2,j)
            end
        end
        delete!(test2, del_list2) #Delete rows that contain illegal elements  
        ypredicted_test2 = predict(linearRegressor2, test2)
        qua_val = maximum(ypredicted_test2)
        push!(qua_max,qua_val)
    end
    
    ymax_ave = sum(lin_max)/seed_numbers   # The average maximum value of linear regression
    println("The average y value for linear regression is: ",ymax_ave)
    push!(ave_lin,ymax_ave)
    ymax_ave2 = sum(qua_max)/seed_numbers
    println("The average y value for quadratic regression is: ",ymax_ave2)
    push!(ave_qua,ymax_ave2)
    lin_max = []
    qua_max = []
    
end

writedlm("Linear.txt",ave_lin)
writedlm("Quadratic.txt",ave_qua)
    

Any[Any["newsvendor", 0.3, 0.4, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 3, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 2, 0.75, 0.75, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.5, 0.1, 0.05, 0.1], Any["newsvendor", 0.3, 0.5, 1, 4, 1, 1, 0.75, 0.75, 0.1, 0.05, 0.1]]
Now parameter setting number is: 1

seed number:1

seed number:2

seed number:3

seed number:4

seed number:5

seed number:6

seed number:7

seed number:8

seed number:9

seed number:10

seed number:11

seed number:12

seed number:13

seed number:14

seed number:15

seed number:16

seed number:17

seed number:18

seed number:19

seed number:20

seed number:21

seed number:22

seed number:23

seed number:24

seed number:25

seed number:26

seed number:27

seed number:28

seed number:29

seed number:30

seed number:31

seed number:32

seed number:33

seed number:34

seed number:35

seed number:36

seed number:37

seed number:38

seed number:39

s

In [40]:
lin_max

250-element Vector{Any}:
 0.7609384059029842
 0.7695117808337442
 0.7872575021030253
 0.7151369161747069
 0.764602864683013
 0.7429976410162815
 0.7553258712587565
 0.7428041300442374
 0.8292082893149954
 0.7008050394725157
 0.7513216171281085
 0.7133722790157244
 0.7306499713469596
 ⋮
 0.7175548697152265
 0.7942273165555879
 0.7835883748621881
 0.7681506158830401
 0.7016539790762244
 0.7762618042481544
 0.7517281084931704
 0.7969686126863677
 0.7657559697536062
 0.7178251105979416
 0.7253971764537851
 0.8113225548277323

In [2]:
fld(9,3)

3

In [1]:
cld(9,2)

5